# PolicyNavigator Agent


##  Install & Import

In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install aixplain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.8/503.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [3]:
import os
from typing import List, Dict, Any
from datetime import datetime
import requests
import logging

os.environ["TEAM_API_KEY"] = "INSERT_HERE"


from aixplain.factories import ToolFactory, ModelFactory, IndexFactory, AgentFactory, TeamAgentFactory

In [ ]:
#CaseExamples_index.delete()
#Regulations_index.delete()

##  Scrape Website & Create Index



In [4]:
from aixplain.modules.model.record import Record

scraped_txt = "/data/dataset/ccpa_enforcement_case_examples.txt"


records = []
index_list = IndexFactory.list(query="CCPA Enforcement Case Examples Index")
for index_1 in index_list["results"]:
  index_1.delete()


CaseExamples_index = IndexFactory.create(
        name="CCPA Enforcement Case Examples",
        description="Index for CCPA Enforcement Case Examples"
    )

scraper = ModelFactory.get("66f423426eb563fa213a3531")
website_result = scraper.run({"text": "https://oag.ca.gov/privacy/ccpa/enforcement"})
text = getattr(website_result, "data", "")
if not text or len(text) < 5:
    raise RuntimeError(f"Scraper returned too little content (len={len(text)})")


with open(scraped_txt, "w", encoding="utf-8") as f:
    f.write(text)

records.append(Record(value=website_result.data))
CaseExamples_index.upsert(records)
CaseExamples_index.count()

print(CaseExamples_index.id)


69269284b0d5bdc79df1e6cd


## Create Index with PDF


In [5]:
records = []
index_list = IndexFactory.list(query="CCPA Regulations Index")
for index_1 in index_list["results"]:
  index_1.delete()

pdf_path = ("/data/dataset/ccpa_regulations.pdf")

docs =[pdf_path]

Regulations_index = IndexFactory.create(
    name="CCPA Regulations Index",
    description="Index for storing parsed PDF content",
    docs=docs,
)

Regulations_index.count()
print(Regulations_index.id)

6926929bb0d5bdc79df1e748


## RAG Agent


In [6]:
regulations_tool = AgentFactory.create_model_tool(model=Regulations_index.id)
cases_tool = AgentFactory.create_model_tool(model=CaseExamples_index.id)

RAG_agent = AgentFactory.create(
    name="RAG Agent",
    description="Answers about regulations and policies using two indexes.",
    instructions=( """
        If the user asks about specific data regulations in the US, use the regulations index.
        If the user asks about situations where the OAG sent a notice of alleged noncompliance and company responses, use the case examples index.
        Cite concise answers based only on retrieved context."""
    ),
    tools=[regulations_tool, cases_tool],
)

/usr/local/lib/python3.12/dist-packages/aixplain/factories/agent_factory/__init__.py:137: UserWarning: Deprecating 'llm_id', use `llm` to define the large language model in agents.
  warnings.warn(


## Legal Research Agent

In [7]:
TAVILY_SEARCH_TOOL = "6736411cf127849667606689"
tavily_tool = ToolFactory.get(TAVILY_SEARCH_TOOL)

research_agent = AgentFactory.create(
    name="Legal Research Agent",
    description="A legal/policy research agent that uses the Tavily web search tool to answer questions about executive orders, regulations, and case law.",
    instructions=( """
You are a legal and policy research assistant. You have one tool: a Tavily web search tool.
 - ALWAYS use this tool to get fresh, grounded information from the web.
 - For questions about executive orders (e.g., 'Is Executive Order 14067 still in effect?'), search specifically for the EO number and official sources like the Federal Register or White House.
 - For questions about case law (e.g., 'Has the CCPA been challenged in court?'), search for relevant court decisions and summarize the most important cases.
 - For general policy questions, search for authoritative sources (government, courts, reputable legal analysis) and synthesize a clear answer.
    After using the tool, ALWAYS base your answer on the search results. Do NOT answer purely from prior knowledge. If search results are unclear or conflicting, say so.
"""
    ),
    tools=[tavily_tool],
)

## Executive Orders Agent

In [8]:
from aixplain.modules.agent.output_format import OutputFormat

csv_path = ("/data/dataset/executiveOrders.csv")

executive_orders_sql_tool = AgentFactory.create_sql_tool(
    name="Executive Orders SQL Tool",
    description="Query and analyze executive orders dataset",
    source=csv_path,
    source_type="csv"
)

executive_orders_agent = AgentFactory.create(
    name="Executive Orders Agent",
    description=(
        """
        Answers questions about U.S. Executive Orders using a structured dataset and web access.
        First, it queries a CSV-backed SQL database of EOs (1994–present), then follows the
        html_url or pdf_url to gather more detail when needed.
        """
    ),
    instructions="""
You are an expert on U.S. Executive Orders.

You have TWO tools:
1) Executive Orders SQL Tool:
   - Source: executiveOrders.csv
   - Columns: citation, document_number, end_page, html_url, pdf_url, type, subtype,
              publication_date, signing_date, start_page, title, disposition_notes,
              executive_order_number
   - Use this tool to locate the relevant executive order(s) and retrieve metadata,
     including the html_url and pdf_url.

2) Tavily web search tool:
   - Use this tool to open and summarize the html_url or pdf_url returned by the SQL query,
     and to gather additional context from the web if needed.

Behavior:
- When the user asks about a specific executive order (e.g., "Executive Order 14067",
  "Tell me more about EO 14067", etc.):
  1. Use the SQL tool to find the row where executive_order_number matches or where
     title or citation clearly refers to that EO.
  2. Extract its key metadata (title, publication_date, signing_date, type, subtype,
     document_number, citation, html_url, pdf_url).
  3. If the user asks for a summary or details, pass the html_url (or pdf_url if needed)
     into the Tavily web search tool to gather the full text or authoritative descriptions.
  4. Then write a clear, concise summary grounded in the webpage/PDF content.

- For more general questions (e.g., "Show me all EOs about cybersecurity since 2010",
  "Which president issued the most EOs about climate?"):
  1. Use the SQL tool to filter and aggregate rows based on years, presidents (if encoded in
     title/citation), topics (search in title or citation), etc.
  2. Present results in a structured way (lists, tables, bullet points).
  3. Optionally, for a small number of key EOs, follow their html_url with Tavily for deeper summaries.

Rules:
- ALWAYS use the SQL tool first to find the relevant executive order(s).
- ONLY then use Tavily on html_url or pdf_url when you need more detail for a summary.
- Do NOT guess. If the dataset does not contain a match, clearly say no matching EO was found.
- When you use Tavily, base your explanation on the content from the returned web pages.
""",
    tools=[executive_orders_sql_tool, tavily_tool],
)

/usr/local/lib/python3.12/dist-packages/aixplain/modules/agent/tool/sql_tool.py:146: UserWarning: Column with dtype 'object' will be stored as TEXT in SQLite
  warnings.warn(f"Column with dtype '{dtype}' will be stored as TEXT in SQLite")
/usr/local/lib/python3.12/dist-packages/aixplain/factories/agent_factory/__init__.py:137: UserWarning: Deprecating 'llm_id', use `llm` to define the large language model in agents.
  warnings.warn(


## Google Docs Agent

In [9]:
GOOGLE_DOCS_TOOL_ID = "INSERT_HERE"

google_docs_tool = ToolFactory.get(GOOGLE_DOCS_TOOL_ID)

CREATE_DOC_MD_ACTION = "GOOGLEDOCS_CREATE_DOCUMENT_MARKDOWN"

google_docs_tool.action_scope = [
    action
    for action in google_docs_tool.actions
    if action.code == CREATE_DOC_MD_ACTION
]

google_docs_agent = AgentFactory.create(
    name="Google Docs Writer Agent",
    description=(
        """Creates a new Google Docs document for any topic or regulation the user chooses,
        using the Composio Google Docs integration."""
    ),
    instructions=f"""
You are a policy/document drafting assistant connected to Google Docs via Composio.

You have ONE external tool: a Google Docs integration, scoped to the action
`{CREATE_DOC_MD_ACTION}`, which creates a new Google Docs document from Markdown.

When the user asks you to create a document (for example: a summary of CCPA,
a brief about a specific executive order, or notes on any regulation):

1. First, plan a clear structure:
   - Use Markdown.
   - Start with a level-1 heading for the document title: "# Title".
   - Use level-2 / level-3 headings for sections and subsections.
   - Use bullet points, numbered lists, and short paragraphs as needed.
   - Aim for 3–10 sections unless the user specifies otherwise.

2. Then call the Google Docs tool with:
   - **title**: a concise document title (e.g., "CCPA Overview",
     "Executive Order 14067 – Summary", "Data Privacy Obligations under CCPA").
   - **markdown_text**: the full Markdown content you drafted.

3. After the tool runs:
   - Read the returned data (document ID, link, etc.).
   - Tell the user that the document was created successfully.
   - Include the document ID or URL if available.
   - Briefly summarize what the document contains (main sections).

If the tool fails or returns an error:
   - Explain that you were unable to create the document.
   - Provide the Markdown content directly in your response so the user can
     copy-paste it into a document manually if needed.

Important:
- ALWAYS generate the Markdown content first, then call the tool.
- Do NOT guess about the tool schema; only use the parameters 'title' and 'markdown_text'
  for the {CREATE_DOC_MD_ACTION} action.
""",
    tools=[google_docs_tool],

)

## Team Agent

In [10]:
team_agent = TeamAgentFactory.create(
    name="Policy Navigator Team Agent",
    description=(
        """A multi-agent system that helps users explore and understand complex government
        regulations (e.g., CCPA), executive orders, and uploaded/linked policies. It can also
        draft structured Google Docs summaries on demand."""
    ),
    instructions="""
You are the top-level coordinator for a team of specialized agents. You NEVER answer
questions yourself and you NEVER use tools directly. Your job is to:

1. Interpret the user's request.
2. Route it to the correct agent—or a sequence of agents.
3. Pass outputs from one agent to another when needed.
4. Return the final result to the user in a clear, structured form.

You orchestrate the following member agents:

──────────────────────────────────────────────────────────
1) RAG Agent
──────────────────────────────────────────────────────────
- Uses a vector index of CCPA regulations, official webpages, and any policy documents
  or URLs that have been indexed.
- Use this for detailed CCPA policy understanding, summaries, definitions, requirements,
  timelines, rights, and obligations.
- Include section references, snippets, or URLs when available.

──────────────────────────────────────────────────────────
2) Executive Orders Agent
──────────────────────────────────────────────────────────
- Uses a SQL tool on executiveOrders.csv (citation, EO number, publication date,
  signing date, html_url, pdf_url, etc.).
- Use this ONLY when the query is specifically about Executive Orders:
    - “Summarize Executive Order 14067.”
    - “What recent EOs relate to cybersecurity?”
- This agent may follow html_url/pdf_url to expand its summary.

──────────────────────────────────────────────────────────
3) Legal Research Agent
──────────────────────────────────────────────────────────
- Uses Tavily to answer broad or external legal/policy research questions.
- Use for queries involving:
    - recent news or enforcement trends
    - comparisons (e.g., CCPA vs GDPR)
    - legal analysis beyond the indexed documents
- Prefer government or authoritative sources.

──────────────────────────────────────────────────────────
4) Google Docs Writer Agent
──────────────────────────────────────────────────────────
- Creates Google Docs via Composio.
- Use ONLY when the user explicitly asks for:
    - “Create a Google Doc…”
    - “Draft a report…”
    - “Turn this into a document…”
- This agent expects Markdown content that you must generate by calling
  RAG / Executive Orders / Legal Research FIRST.

──────────────────────────────────────────────────────────
ROUTING LOGIC (CRITICAL)
──────────────────────────────────────────────────────────

1. DOCUMENT CREATION (highest priority)
   If the user asks to:
   - create, write, draft, build, or generate a *Google Doc*
   - create a *report*, *summary document*, *policy brief*, or *write-up*

   → FIRST determine what content the doc should contain.
   → Use one of:
        • RAG Agent (indexed policies like CCPA)
        • Executive Orders Agent (EO-specific content)
        • Legal Research Agent (broad or external context)
   → THEN pass the resulting summary to the Google Docs Writer Agent to create the doc.

   Example:
     User: “Create a Google Doc summarizing the CCPA.”
     Steps:
        1. Call RAG Agent → get CCPA summary.
        2. Send that summary to Google Docs Writer Agent → create doc.
        3. Return the document link.

2. EXECUTIVE ORDER QUESTIONS
   If the query mentions:
   - “Executive Order”, “EO”, “EO ####”, “order number”, or any specific EO topic

   → ALWAYS route to the Executive Orders Agent.

3. INDEXED POLICY QUESTIONS (CCPA)
   If the question is about:
   - CCPA
   - Any uploaded PDF or indexed link
   - “this policy”, “this regulation”, “indexed document”, etc.

   → Route to the RAG Agent.

4. BROAD / GENERAL / WEB-REQUIRED QUESTIONS
   If the question references:
   - “recent”, “latest”, “trends”, “how do courts interpret…”
   - cross-state comparisons
   - broad legal topics not in the index

   → Route to the Legal Research Agent.

──────────────────────────────────────────────────────────
AGENT-TO-AGENT COORDINATION RULE
──────────────────────────────────────────────────────────

When assembling a multi-step workflow:

- You MAY call one agent to gather content,
  THEN call another agent with that content.

- You MUST pass the exact output of the first agent into the second one
  with minimal modification (only formatting into Markdown is allowed).

- You MUST clearly decide which agent to call next.

──────────────────────────────────────────────────────────
ANSWERING STYLE
──────────────────────────────────────────────────────────
- Never apologize or say you lack access to tools — delegate instead.
- Always structure answers clearly using bullets, headings, timelines, or tables.
- Include citations, URLs, section references, or EO metadata when available.
- Be thorough but concise.
- Always return the final answer from the delegated agent(s).
""",
    agents=[
        RAG_agent,
        executive_orders_agent,
        research_agent,
        google_docs_agent,
    ],
)

## Testing Agent


In [13]:
response1 = team_agent.run(data={"query": "Create a Google Doc that summarizes the key points of the CCPA and give me its url"})
print(response1.data.output)

🤖  Policy Navigator Team Agent | ⏳ Starting agent execution...
⚙️  Mentalist | GPT-4o Mini | ⏳ Agent 'Mentalist' is making LLM call: GPT-4o Mini
⚙️  RAG Agent | GPT-4o Mini | ⏳ Agent 'RAG Agent' is making LLM call: GPT-4o Mini
⚙️  RAG Agent | search-aixplain-ccpa_regulations_index | ⏳ Agent 'RAG Agent' is executing tool: search-aixplain-ccpa_regulations_index
⚙️  RAG Agent | GPT-4o Mini | ⏳ Agent 'RAG Agent' is making LLM call: GPT-4o Mini
⚙️  RAG Agent | search-aixplain-ccpa_regulations_index | ✓ No data returned from the search model.
⚙️  RAG Agent | GPT-4o Mini | ⏳ Agent 'RAG Agent' is making LLM call: GPT-4o Mini
⚙️  RAG Agent | search-aixplain-ccpa_enforcement_case_examples | ⏳ Agent 'RAG Agent' is executing tool: search-aixplain-ccpa_enforcement_case_examples
⚙️  RAG Agent | GPT-4o Mini | ⏳ Agent 'RAG Agent' is making LLM call: GPT-4o Mini
⚙️  Google Docs Writer Agent | GPT-4o Mini | ⏳ Agent 'Google Docs Writer Agent' is making LLM call: GPT-4o Mini
⚙️  Google Docs Writer Agent |

In [14]:
response1.data["intermediate_steps"]

[{'agent': 'mentalist',
  'input': 'Create a Google Doc that summarizes the key points of the CCPA and give me its url',
  'output': ['{"name":"Task 1","step_id":"1","description":"Gather key points of the CCPA","expectedOutput":"","dependencies":[],"agent":"RAG Agent","status":"PENDING"}',
   '{"name":"Task 2","step_id":"2","description":"Create a Google Doc with the CCPA summary","expectedOutput":"","dependencies":[{"name":"Task 1","step_id":"1","description":"Gather key points of the CCPA","expectedOutput":"","dependencies":[],"agent":"RAG Agent","status":"PENDING"}],"agent":"Google Docs Writer Agent","status":"PENDING"}'],
  'tool_steps': None,
  'llm_steps': None,
  'thought': 'Planning: Created 2 steps to solve the problem. The user has requested a Google Doc summarizing the key points of the California Consumer Privacy Act (CCPA). I will first call the RAG Agent to gather the necessary information about the CCPA, and then I will pass that information to the Google Docs Writer Ag

In [18]:
response2 = team_agent.run(data={"query": "How many Executive Orders were signed in 2017"})
print(response2.data.output)

🤖  Policy Navigator Team Agent | ⏳ Starting agent execution...
⚙️  Mentalist | GPT-4o Mini | ⏳ Agent 'Mentalist' is making LLM call: GPT-4o Mini
🤖  Orchestrator | ⏳ Orchestrator assigned work to: Executive Orders Agent - Task 1: Query the executive orders dataset t
⚙️  Executive Orders Agent | GPT-4o Mini | ⏳ Agent 'Executive Orders Agent' is making LLM call: GPT-4o Mini
🤖  Response Generator | ⏳ Generating final response...
✅ Done | (22.8 s total | 5 API calls | $0.0040251)
There were 60 Executive Orders signed in 2017.


In [19]:
response2.data["intermediate_steps"]

[{'agent': 'mentalist',
  'input': 'How many Executive Orders were signed in 2017',
  'output': ['{"name":"Task 1","step_id":"1","description":"Query the executive orders dataset to count the number of Executive Orders signed in 2017.","expectedOutput":"","dependencies":[],"agent":"Executive Orders Agent","status":"PENDING"}'],
  'tool_steps': None,
  'llm_steps': None,
  'thought': "Planning: Created 1 step to solve the problem. The user's request is specifically about Executive Orders and requires a count of those signed in the year 2017. Therefore, I will route this request to the Executive Orders Agent, which has the capability to query the executive orders dataset for the relevant information.",
  'runTime': 2.576,
  'usedCredits': 0.0009767999999999999,
  'apiCalls': 1,
  'task': 'Create execution plan',
  'action': None,
  'reason': "Planning: Created 1 step to solve the problem. The user's request is specifically about Executive Orders and requires a count of those signed in th

In [22]:
response3 = team_agent.run(data={"query": "Has Section 230 ever been challenged in court? What was the outcome?”"})
print(response3.data.output)

🤖  Policy Navigator Team Agent | ⏳ Starting agent execution...
🤖  Mentalist | ⏳ Allocating subtasks to available agents
⚙️  Mentalist | GPT-4o Mini | ⏳ Agent 'Mentalist' is making LLM call: GPT-4o Mini
⚙️  Legal Research Agent | GPT-4o Mini | ⏳ Agent 'Legal Research Agent' is making LLM call: GPT-4o Mini
⚙️  Legal Research Agent | Tavily Search API | ⏳ Agent 'Legal Research Agent' is executing tool: Tavily Search API
⚙️  Legal Research Agent | GPT-4o Mini | ⏳ Agent 'Legal Research Agent' is making LLM call: GPT-4o Mini
⚙️  Legal Research Agent | Tavily Search API | ⏳ Agent 'Legal Research Agent' is executing tool: Tavily Search API
⚙️  Legal Research Agent | GPT-4o Mini | ⏳ Agent 'Legal Research Agent' is making LLM call: GPT-4o Mini
⚙️  Response Generator | GPT-4o Mini | ⏳ Agent 'response_generator' is making LLM call: GPT-4o Mini
✅ Done | (23.6 s total | 6 API calls | $0.0035263499999999993)
In recent court cases challenging Section 230, two notable outcomes are as follows:

1. **Gon

In [23]:
response4 = team_agent.run(data={"query": "Draft a one-page brief explaining Executive Order 14067."})
print(response4.data.output)

🤖  Policy Navigator Team Agent | ⏳ Starting agent execution...
⚙️  Mentalist | GPT-4o Mini | ⏳ Agent 'Mentalist' is making LLM call: GPT-4o Mini
🤖  Orchestrator | ⏳ Orchestrator assigned work to: Executive Orders Agent - Task 1: Retrieve details and summary of Exec
⚙️  Executive Orders Agent | GPT-4o Mini | ⏳ Agent 'Executive Orders Agent' is making LLM call: GPT-4o Mini
⚙️  Google Docs Writer Agent | GPT-4o Mini | ⏳ Agent 'Google Docs Writer Agent' is making LLM call: GPT-4o Mini
⚙️  Response Generator | GPT-4o Mini | ⏳ Agent 'response_generator' is making LLM call: GPT-4o Mini
✅ Done | (60.0 s total | 8 API calls | $0.00875895)
The document has been created successfully. You can view it [here](https://docs.google.com/document/d/1m50VzsY3Y6LYw9Pwhb_pTkOBeCq-VUITXsexXeVt5gw). It contains the following sections:

- **Introduction**: Overview of Executive Order 14067, its signing date, and purpose.
- **Key Points**: Highlights including public input, regulatory framework, ban on CBDCs, a